In [3]:
import time, copy, sys

sys.path.append(r'C:\Users\gavin\OneDrive\Desktop\Artificial Intelligence')
from sudoku_parser import SudokuPuzzle

# load up the sudoku puzzle
my_sudoku = SudokuPuzzle(r'C:\Users\gavin\OneDrive\Desktop\Artificial Intelligence\Med-P2.txt')
my_sudoku.set_domains()
my_sudoku.display()

# find all the cells around a given one
def get_neighbors(r, c):
    nearby = set()
    # same row and column
    for i in range(9):
        nearby.add((r, i))
        nearby.add((i, c))
    # also in the same box
    box_start_r, box_start_c = 3*(r//3), 3*(c//3)
    for rr in range(box_start_r, box_start_r+3):
        for cc in range(box_start_c, box_start_c+3):
            nearby.add((rr, cc))
    nearby.discard((r,c))
    return nearby

# pick the next cell to try
def pick_next_cell(board):
    empties = board.available_indexes()
    best_cell = None
    highest_deg = -1
    for r, c in empties:
        deg = 0
        for nr, nc in get_neighbors(r, c):
            if board.grid[nr][nc] == 0:
                deg += 1
        if deg > highest_deg:
            highest_deg = deg
            best_cell = (r, c)
    return best_cell

# remove impossible numbers from neighbors
def forward_check(board, row, col, num):
    for r, c in get_neighbors(row, col):
        if board.grid[r][c] == 0 and board.domains[r][c]:
            if num in board.domains[r][c]:
                board.domains[r][c].remove(num)
                if not board.domains[r][c]:
                    return False
    return True

# the backtracking + forward check solver
def solve_sudoku(board):
    if not board.available_indexes():  # no empties left
        return True

    row, col = pick_next_cell(board)
    options = board.domains[row][col] or list(range(1,10))

    for n in options:
        board.set_digit(row, col, n)
        if board.is_valid():
            # save state in case we need to undo
            backup_grid = copy.deepcopy(board.grid)
            backup_domains = copy.deepcopy(board.domains)

            if not forward_check(board, row, col, n):
                board.grid = backup_grid
                board.domains = backup_domains
                board.set_digit(row, col, 0)
                continue

            if solve_sudoku(board):
                return True

            # undo if dead end
            board.grid = backup_grid
            board.domains = backup_domains
            board.set_digit(row, col, 0)
        else:
            board.set_digit(row, col, 0)

    return False  # stuck here

# ---- run it ----
my_sudoku = SudokuPuzzle(r'C:\Users\gavin\OneDrive\Desktop\Artificial Intelligence\Med-P2.txt')
my_sudoku.set_domains()

if solve_sudoku(my_sudoku):
    print("\nsolved!")
    my_sudoku.display()
else:
    print("\couldn't solve")


? 7 9 4 3 ? 1 ? ?
3 ? ? ? ? ? 6 ? ?
8 5 ? ? 6 7 ? 3 ?
? ? ? ? ? 6 ? 8 ?
? ? 5 ? 4 ? 7 ? ?
? 8 ? 5 ? ? ? ? ?
? 9 ? 1 7 ? ? 6 3
? ? 8 ? ? ? ? ? 1
? ? 6 ? 9 4 2 7 ?

finally got it solved!
6 7 9 4 3 5 1 2 8
3 4 2 9 8 1 6 5 7
8 5 1 2 6 7 9 3 4
4 1 3 7 2 6 5 8 9
9 6 5 3 4 8 7 1 2
2 8 7 5 1 9 3 4 6
5 9 4 1 7 2 8 6 3
7 2 8 6 5 3 4 9 1
1 3 6 8 9 4 2 7 5
